# Getting started with our ultimate beginner guide!

## This tutorial will walk you through the basics of using the `usb` lighting package. Let's get started by training a FixMatch model on CIFAR-10!

In [1]:
import semilearn
from semilearn import get_dataset, get_data_loader, get_net_builder, get_algorithm, get_config, Trainer


/media/Zeus/haoc/miniconda/envs/test_semilearn_031/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1: define configs and create config

In [2]:
config = {
    'algorithm': 'fixmatch',
    'net': 'vit_tiny_patch2_32',
    'use_pretrain': True, 
    'pretrain_path': 'https://github.com/microsoft/Semi-supervised-learning/releases/download/v.0.0.0/vit_tiny_patch2_32_mlp_im_1k_32.pth',

    # optimization configs
    'epoch': 1,  
    'num_train_iter': 5000,  
    'num_eval_iter': 500,  
    'num_log_iter': 50,  
    'optim': 'AdamW',
    'lr': 5e-4,
    'layer_decay': 0.5,
    'batch_size': 16,
    'eval_batch_size': 16,


    # dataset configs
    'dataset': 'cifar10',
    'num_labels': 40,
    'num_classes': 10,
    'img_size': 32,
    'crop_ratio': 0.875,
    'data_dir': './data',
    'ulb_samples_per_class': None,

    # algorithm specific configs
    'hard_label': True,
    'uratio': 2,
    'ulb_loss_ratio': 1.0,

    # device configs
    'gpu': 0,
    'world_size': 1,
    'distributed': False,
    "num_workers": 2,
}
config = get_config(config)

/bin/sh: 1: netstat: not found


## Step 2: create model and specify algorithm

In [3]:
algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler


## Step 3: create dataset

In [4]:
dataset_dict = get_dataset(config, config.algorithm, config.dataset, config.num_labels, config.num_classes, data_dir=config.data_dir, include_lb_to_ulb=config.include_lb_to_ulb)
train_lb_loader = get_data_loader(config, dataset_dict['train_lb'], config.batch_size)
train_ulb_loader = get_data_loader(config, dataset_dict['train_ulb'], int(config.batch_size * config.uratio))
eval_loader = get_data_loader(config, dataset_dict['eval'], config.eval_batch_size)

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


## Step 4: train

In [5]:
trainer = Trainer(config, algorithm)
trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)

Epoch: 0
50 iteration USE_EMA: True, train/sup_loss: 1.9210, train/unsup_loss: 0.0000, train/total_loss: 1.9210, train/util_ratio: 0.0000, train/run_time: 0.1366, lr: 0.0005, train/prefecth_time: 0.0040 
100 iteration USE_EMA: True, train/sup_loss: 0.9313, train/unsup_loss: 0.0000, train/total_loss: 0.9313, train/util_ratio: 0.0000, train/run_time: 0.1369, lr: 0.0005, train/prefecth_time: 0.0037 
150 iteration USE_EMA: True, train/sup_loss: 0.4322, train/unsup_loss: 0.0000, train/total_loss: 0.4322, train/util_ratio: 0.0000, train/run_time: 0.1365, lr: 0.0005, train/prefecth_time: 0.0037 
200 iteration USE_EMA: True, train/sup_loss: 0.2618, train/unsup_loss: 0.0000, train/total_loss: 0.2618, train/util_ratio: 0.0000, train/run_time: 0.1383, lr: 0.0005, train/prefecth_time: 0.0040 
250 iteration USE_EMA: True, train/sup_loss: 0.4954, train/unsup_loss: 0.0902, train/total_loss: 0.5856, train/util_ratio: 0.0938, train/run_time: 0.1384, lr: 0.0005, train/prefecth_time: 0.0038 
300 iteratio

/media/Zeus/haoc/miniconda/envs/test_semilearn_031/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


confusion matrix:
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
model saved: ./saved_models/fixmatch/latest_model.pth
model saved: ./saved_models/fixmatch/model_best.pth
500 iteration, USE_EMA: True, train/sup_loss: 0.0236, train/unsup_loss: 0.0640, train/total_loss: 0.0877, train/util_ratio: 0.2812, train/run_time: 0.1395, eval/loss: 5.5914, eval/top-1-acc: 0.1000, eval/balanced_acc: 0.1000, eval/precision: 0.0100, eval/recall: 0.1000, eval/F1: 0.0182, lr: 0.0005, train/prefecth_time: 0.0038 BEST_EVAL_ACC: 0.1000, at 500 iters
550 iteration USE_EMA: True, train/sup_loss: 0.0866, train/unsup_loss: 0.2069, train/total_loss: 0.2936, train/util_ratio: 0.2812, train/run_time: 0.1393, lr: 0.0005, train/prefecth

/media/Zeus/haoc/miniconda/envs/test_semilearn_031/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


confusion matrix:
[[0.127 0.    0.    0.    0.484 0.    0.367 0.007 0.004 0.011]
 [0.    0.277 0.    0.    0.568 0.    0.049 0.039 0.    0.067]
 [0.    0.    0.    0.    0.93  0.    0.066 0.004 0.    0.   ]
 [0.    0.    0.    0.002 0.92  0.    0.065 0.012 0.    0.001]
 [0.    0.    0.    0.    0.995 0.    0.003 0.002 0.    0.   ]
 [0.    0.    0.    0.    0.955 0.002 0.034 0.009 0.    0.   ]
 [0.    0.    0.    0.    0.445 0.    0.555 0.    0.    0.   ]
 [0.    0.    0.    0.    0.847 0.    0.003 0.15  0.    0.   ]
 [0.002 0.    0.    0.    0.822 0.    0.142 0.009 0.014 0.011]
 [0.    0.    0.    0.    0.446 0.    0.022 0.005 0.    0.527]]
model saved: ./saved_models/fixmatch/latest_model.pth
model saved: ./saved_models/fixmatch/model_best.pth
1000 iteration, USE_EMA: True, train/sup_loss: 0.0011, train/unsup_loss: 0.2126, train/total_loss: 0.2137, train/util_ratio: 0.7500, train/run_time: 0.1401, eval/loss: 2.2794, eval/top-1-acc: 0.2649, eval/balanced_acc: 0.2649, eval/precision: 0.

[2023-07-19 23:13:01,538 INFO] confusion matrix
[2023-07-19 23:13:01,540 INFO] [[0.956 0.004 0.    0.001 0.    0.    0.    0.    0.022 0.017]
 [0.    0.99  0.    0.    0.    0.    0.    0.    0.    0.01 ]
 [0.046 0.    0.77  0.01  0.134 0.016 0.018 0.004 0.001 0.001]
 [0.003 0.002 0.002 0.872 0.015 0.079 0.023 0.    0.002 0.002]
 [0.    0.    0.002 0.007 0.949 0.007 0.008 0.026 0.001 0.   ]
 [0.    0.    0.001 0.018 0.005 0.965 0.002 0.009 0.    0.   ]
 [0.002 0.    0.003 0.001 0.    0.002 0.991 0.    0.001 0.   ]
 [0.004 0.    0.002 0.005 0.005 0.03  0.    0.951 0.001 0.002]
 [0.006 0.008 0.    0.    0.    0.    0.    0.    0.977 0.009]
 [0.003 0.034 0.    0.    0.    0.    0.    0.    0.001 0.962]]
[2023-07-19 23:13:01,542 INFO] evaluation metric
[2023-07-19 23:13:01,543 INFO] acc: 0.9383
[2023-07-19 23:13:01,543 INFO] precision: 0.9409
[2023-07-19 23:13:01,544 INFO] recall: 0.9383
[2023-07-19 23:13:01,544 INFO] f1: 0.9375
[2023-07-19 23:13:02,617 INFO] Best acc 0.9387 at epoch 0
[20

model saved: ./saved_models/fixmatch/latest_model.pth


## Step 5: evaluate

In [6]:
trainer.evaluate(eval_loader)

[2023-07-19 23:14:07,511 INFO] confusion matrix
[2023-07-19 23:14:07,513 INFO] [[0.956 0.004 0.    0.001 0.    0.    0.    0.    0.022 0.017]
 [0.    0.99  0.    0.    0.    0.    0.    0.    0.    0.01 ]
 [0.046 0.    0.77  0.01  0.134 0.016 0.018 0.004 0.001 0.001]
 [0.003 0.002 0.002 0.872 0.015 0.079 0.023 0.    0.002 0.002]
 [0.    0.    0.002 0.007 0.949 0.007 0.008 0.026 0.001 0.   ]
 [0.    0.    0.001 0.018 0.005 0.965 0.002 0.009 0.    0.   ]
 [0.002 0.    0.003 0.001 0.    0.002 0.991 0.    0.001 0.   ]
 [0.004 0.    0.002 0.005 0.005 0.03  0.    0.951 0.001 0.002]
 [0.006 0.008 0.    0.    0.    0.    0.    0.    0.977 0.009]
 [0.003 0.034 0.    0.    0.    0.    0.    0.    0.001 0.962]]
[2023-07-19 23:14:07,515 INFO] evaluation metric
[2023-07-19 23:14:07,516 INFO] acc: 0.9383
[2023-07-19 23:14:07,517 INFO] precision: 0.9409
[2023-07-19 23:14:07,517 INFO] recall: 0.9383
[2023-07-19 23:14:07,518 INFO] f1: 0.9375


{'acc': 0.9383,
 'precision': 0.9408766240300274,
 'recall': 0.9382999999999999,
 'f1': 0.9375322532701483}

## Step 6: predict

In [7]:
y_pred, y_logits = trainer.predict(eval_loader)